<a href="https://colab.research.google.com/github/mxn04420/Assignment-5/blob/main/NNDL_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from google.colab import drive

drive.mount('/content/gdrive')
dataset = pd.read_csv('/content/gdrive/My Drive/Sentiment.csv')

mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)

print(score)
print(acc)

Mounted at /content/gdrive


<ipython-input-2-ba84f9750b30>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-2-ba84f9750b30>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


291/291 - 64s - loss: 0.8320 - accuracy: 0.6431 - 64s/epoch - 221ms/step
144/144 - 3s - loss: 0.7470 - accuracy: 0.6732 - 3s/epoch - 22ms/step
0.746999979019165
0.6732197403907776


In [ ]:
from keras.models import load_model

#saving and loading the model
model.save('sentimentAnalysis.h5')
model= load_model('sentimentAnalysis.h5')

#predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0]
sentiment = np.argmax(sentiment_probs)

#checking the sentiment of the sample data
print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 1s - 541ms/epoch - 541ms/step
[0.567826   0.14409716 0.28807685]
Neutral


In [6]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#GridSearch model is used
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20, 40]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<ipython-input-6-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 125s - loss: 0.8302 - accuracy: 0.6402 - 125s/epoch - 169ms/step
186/186 - 3s - loss: 0.7499 - accuracy: 0.6756 - 3s/epoch - 15ms/step
744/744 - 115s - loss: 0.8252 - accuracy: 0.6480 - 115s/epoch - 154ms/step
186/186 - 4s - loss: 0.7724 - accuracy: 0.6606 - 4s/epoch - 20ms/step
744/744 - 115s - loss: 0.8273 - accuracy: 0.6460 - 115s/epoch - 155ms/step
186/186 - 3s - loss: 0.7921 - accuracy: 0.6573 - 3s/epoch - 15ms/step
744/744 - 112s - loss: 0.8251 - accuracy: 0.6463 - 112s/epoch - 151ms/step
186/186 - 3s - loss: 0.7647 - accuracy: 0.6722 - 3s/epoch - 15ms/step
744/744 - 112s - loss: 0.8155 - accuracy: 0.6469 - 112s/epoch - 151ms/step
186/186 - 5s - loss: 0.7830 - accuracy: 0.6679 - 5s/epoch - 24ms/step
Epoch 1/2
744/744 - 109s - loss: 0.8234 - accuracy: 0.6517 - 109s/epoch - 147ms/step
Epoch 2/2
744/744 - 108s - loss: 0.6871 - accuracy: 0.7058 - 108s/epoch - 145ms/step
186/186 - 3s - loss: 0.7367 - accuracy: 0.6848 - 3s/epoch - 16ms/step
Epoch 1/2
744/744 - 108s - loss: 0.